In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Extract dataset name from the file path
import os
dataset_path = '/mnt/ceph/users/lbrown/MouseData/David4EPI/LineageGraph.json'
dataset_name = os.path.basename(os.path.dirname(dataset_path))
print(f"Dataset name: {dataset_name}")

with open(dataset_path, 'r') as f:
    data = json.load(f)

Dataset name: David4EPI


In [ ]:
nodes_data = data['Nodes']
edges_data = data['Edges']



# Create a dictionary for node properties
nodes = {node['Name']: {'frame': int(node['Name'].split('_')[0])} for node in nodes_data}

# Build parent-child relationships
parent_of = {edge['EndNodes'][1]: edge['EndNodes'][0] for edge in edges_data}
children_of = {}
for edge in edges_data:
    parent_name = edge['EndNodes'][0]
    child_name = edge['EndNodes'][1]
    if parent_name not in children_of:
        children_of[parent_name] = []
    children_of[parent_name].append(child_name)

# Identify roots (nodes with no parents)
root_nodes = sorted([name for name in nodes if name not in parent_of])

pos = {}
x_offset = 0

def layout_tree(node_name, current_x):
    """Recursively assign x positions to nodes."""
    global x_offset
    
    if node_name in pos:
        return

    pos[node_name] = (current_x, nodes[node_name]['frame'])

    children = children_of.get(node_name, [])
    
    if not children:
        x_offset += 1 
        return
        
    start_x = current_x - (len(children) - 1) / 2.0
    
    child_positions = []
    for i, child in enumerate(sorted(children)):
        temp_x = start_x + i
        is_occupied = any(
            p[1] == nodes[child]['frame'] and abs(p[0] - temp_x) < 0.5
            for p in pos.values()
        )
        if is_occupied:
             temp_x = x_offset + 1

        layout_tree(child, temp_x)
        child_positions.append(pos[child][0])

    if child_positions:
        pos[node_name] = (np.mean(child_positions), nodes[node_name]['frame'])

# Layout all trees
for root in root_nodes:
    if root not in pos:
        layout_tree(root, x_offset)


plt.figure(figsize=(30, 22))

# Plot edges
for child, parent in parent_of.items():
     if parent in pos and child in pos:
        plt.plot([pos[parent][0], pos[child][0]], [pos[parent][1], pos[child][1]], 'k-', lw=0.7, alpha=0.6)

# Plot nodes and labels
for name, p in pos.items():
    color = 'lightblue' 
    if name in children_of:
        if len(children_of[name]) > 1:
            color = 'green'
    else:
        color = 'red'

    plt.plot(p[0], p[1], 'o', markersize=7, color=color, markeredgecolor='black', markeredgewidth=0.5)
    plt.text(p[0] + 0.1, p[1], name, fontsize=5, ha='left', va='center')
    
plt.xlabel("Individual Cell Lineages", fontsize=16)
plt.ylabel("Frame Number", fontsize=16)
plt.title(f"Cell Lineage Tree with Node Labels - {dataset_name}", fontsize=20)
plt.gca().invert_yaxis()
plt.xticks([]) 
plt.grid(axis='y', linestyle='--', linewidth=0.5)

# Create a legend
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Stasis (Continues)', markerfacecolor='lightblue', markersize=12),
                   Line2D([0], [0], marker='o', color='w', label='Mitosis (Division)', markerfacecolor='green', markersize=12),
                   Line2D([0], [0], marker='o', color='w', label='Death / End of Track', markerfacecolor='red', markersize=12)]

plt.legend(handles=legend_elements, loc='upper right', fontsize=14)
plt.tight_layout()

# Use dataset name in the filename
output_filename = f'lineage_tree_with_labels_{dataset_name}.svg'
plt.savefig(output_filename, dpi=800, bbox_inches='tight')
plt.close()

print(f"Lineage tree with node labels saved as {output_filename}")

Lineage tree with node labels saved as lineage_tree_with_labels.png


In [6]:
# frame, nuclues

#   203   325     0     1
#   205   333     0     1
#     201    97     0     1
#   201   321     0     1
#     191   296     0     1
#       180   286     0     1
#         169   279     0     1
#           159   133     0     1
pairs = [
    (203, 325),
    (205, 333),
    (201, 97),
    (201, 321),
    (191, 296),
    (180, 286),
    (169, 279),
    (159, 133)
]

In [7]:
def getfilename(frame):
    """Generate a filename based on the frame number."""
    return f"label_reg8_{frame}.tif"

In [ ]:
from tifffile import imread
import numpy as np

for frame, nucleus in pairs:
    filename = getfilename(frame)
    filepath = os.path.join(
        "/mnt/home/dchhantyal/3d-cnn-classification/raw-data/221016_FUCCI_Nanog_stack_3/registered_label_images",
        filename,
    )

    if os.path.exists(filepath):
        image = imread(filepath)
        unique_nuclei = np.unique(image)
        present = nucleus in unique_nuclei

        print(f"\nFrame {frame}:")
        print(f"  - Nucleus requested: {nucleus}")
        print(f"  - Nuclei present in image: {unique_nuclei.tolist()}")
        if present:
            print(f"  ✔ Nucleus {nucleus} is PRESENT in the image.")
        else:
            print(f"  ✘ Nucleus {nucleus} is MISSING in the image.")

        plt.figure(figsize=(10, 10))
        plt.imshow(image.max(axis=0), cmap='gray')
        plt.title(f"Frame: {frame}, Nucleus: {nucleus}")
        plt.axis('off')
        plt.close()
    else:
        print(f"\nFrame {frame}:")
        print(f"  - File {filepath} does not exist.")


Frame 203:
  - Nucleus requested: 325
  - Nuclei present in image: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 100, 102, 103, 244, 245, 246]
  ✘ Nucleus 325 is MISSING in the image.

Frame 205:
  - Nucleus requested: 333
  - Nuclei present in image: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 101, 253]
  ✘ Nucleus 333 is MISSING in the image.
